In [16]:
import pandas as pd

cdc_mask = pd.read_csv('Coronovarius_Maia_Angel_Marie/CDC_Mask/cdc_mask.csv')

In [17]:
cdc_mask.shape

(36254, 32)

In [18]:
cdc_mask.head()

,date_only,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,publish_date,retweet_count,retweet_or_quote_id,...,user_id,user_name,user_screen_name,user_statuses_count,user_verified,datetime_EST,date_EST,week_number,month,segment
0,2020-01-22 00:00:00,1,NaN,1219819938468155392,NaN,NaN,NaN,2020-01-22 03:11:51,0,1.219773e+18,...,544714500,Tink 💿(Gwth's Girl) TinkerPixie,PixieWings76,3213,False,2020-01-21 22:11:51-05:00,2020-01-21,1,1,1
1,2020-01-22 00:00:00,0,NaN,1219855944906993664,NaN,NaN,NaN,2020-01-22 05:34:55,0,NaN,...,14076534,jmarine,jmarine,47186,False,2020-01-22 00:34:55-05:00,2020-01-22,1,1,1
2,2020-01-22 00:00:00,1,NaN,1219950051868454912,DeloarDuda,1.219950e+18,5.479425e+07,2020-01-22 11:48:52,0,NaN,...,131845620,Popsy Kanagaratnam,PKanagaratnam,259665,False,2020-01-22 06:48:52-05:00,2020-01-22,1,1,1
3,2020-01-22 00:00:00,1,NaN,1219953367503556608,SariArhoHavren,1.219951e+18,1.135341e+09,2020-01-22 12:02:03,0,NaN,...,24993262,Thomas//孔天慧,takiwatanga,7974,False,2020-01-22 07:02:03-05:00,2020-01-22,1,1,1
4,2020-01-22 00:00:00,1,NaN,1219971183187070977,Alexand64731014,1.219875e+18,2.231528e+09,2020-01-22 13:12:50,0,NaN,...,19329334,幸福大街,passby2009,3271,False,2020-01-22 08:12:50-05:00,2020-01-22,1,1,1


In [19]:
cdc_mask.columns

Index(['date_only', 'favorite_count', 'hashtags', 'id',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'publish_date', 'retweet_count',
       'retweet_or_quote_id', 'retweet_or_quote_screen_name',
       'retweet_or_quote_user_id', 'sent_comp', 'sent_comp_w', 'text_mod',
       'tweet_type', 'tweet_type_2', 'tweet_url', 'urls', 'user_created_at',
       'user_followers_count', 'user_friends_count', 'user_id', 'user_name',
       'user_screen_name', 'user_statuses_count', 'user_verified',
       'datetime_EST', 'date_EST', 'week_number', 'month', 'segment'],
      dtype='object')

In [20]:
# Filter out prior to text list
pho_cdc = ["cdc", "c\.d\.c", "centers for disease control", "center for disease control"]
pho_cdc_acctnames = ["cdcgov", "cdcemergency"]
mask_3 = ["mask", "n95", "bandana", "scarf", "cloth face", "cloth cover", "face cover"]

all_cdc_mask_terms = pho_cdc + pho_cdc_acctnames + mask_3

In [25]:
import nltk
import re
from nltk.collocations import *

import string

# NLTK stopwords
stopword = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

def preprocess_tweet_basic(tweet):
    # remove any RT/FAV - might not be needed, also need to double check regex expression
    # tweet = re.sub(r'^(rt|RT|FAV)', '', tweet)

    tweet = tweet.lower()

    # remove usernames
    tweet = re.sub('@[^\s]+', '', tweet)
    # remove the #hashtag
    tweet = re.sub('#[^\s]+', '', tweet)

    # remove punctuation
    nopunc = [char for char in tweet if char not in string.punctuation]
    # Join the characters again to form the string.
    tweet = ''.join(nopunc)
    
    # Replace "face mask" and "face masks" and "face-mask" and "face-masks" and "facemasks" by "facemask" (added by Marie)
    tweet = tweet.replace("face mask", "facemask")
    tweet = tweet.replace("face masks", "facemask")
    tweet = tweet.replace("face-mask", "facemask")
    tweet = tweet.replace("face-masks", "facemask")
    tweet = tweet.replace("facemasks", "facemask")

    # remove numbers - commented out incase we want to capture things like n95 mask
    # tweet = re.sub('[0-9]+', '', tweet)

    tweet = re.split('\W+', tweet)

    # lemmatization
    tweet = [wn.lemmatize(word) for word in tweet]
    tweet = [word for word in tweet if word not in stopword]

    # remove any single letter words
    tweet = [word for word in tweet if len(word) != 1]

    # remove any single digits (since we didnt get rid of all numbers)
    # not needed if numbers are removed.
    tweet = [x for x in tweet if not x.isdigit()]
    
    # remove terms in the query (added by Marie)
    tweet = [x for x in tweet if x not in all_cdc_mask_terms]

    str_list = list(filter(None, tweet))

    final_text = ' '.join(str_list)
    # print(str_list)
    return final_text

In [26]:
cdc_mask['text_tokens'] = cdc_mask['text_mod'].apply(preprocess_tweet_basic)

In [27]:
cdc_mask['text_tokens'][0]

'well go new protocol work rural hospital testing er going flooded scared people wanting tested test run speed dial huzzah night shift protect folk'

In [28]:
cdc_mask['week_number'].value_counts()

11    7624
6     6774
12    3845
7     2957
10    2042
15    1998
13    1714
4     1458
2     1289
16    1261
9     1152
5     1124
3      974
14     916
1      731
8      395
Name: week_number, dtype: int64

In [29]:
cdc_mask['month'].value_counts()

4    14147
2     9439
3     8785
5     1994
1     1889
Name: month, dtype: int64

In [30]:
cdc_mask_tweets_month_1 = cdc_mask[cdc_mask['month'] == 1].reset_index()
cdc_mask_tweets_month_2 = cdc_mask[cdc_mask['month'] == 2].reset_index()
cdc_mask_tweets_month_3 = cdc_mask[cdc_mask['month'] == 3].reset_index()
cdc_mask_tweets_month_4 = cdc_mask[cdc_mask['month'] == 4].reset_index()
cdc_mask_tweets_month_5 = cdc_mask[cdc_mask['month'] == 5].reset_index()

In [31]:
cdc_mask_tweets_week_1 = cdc_mask[cdc_mask['week_number'] == 1].reset_index()
cdc_mask_tweets_week_2 = cdc_mask[cdc_mask['week_number'] == 2].reset_index()
cdc_mask_tweets_week_3 = cdc_mask[cdc_mask['week_number'] == 3].reset_index()
cdc_mask_tweets_week_4 = cdc_mask[cdc_mask['week_number'] == 4].reset_index()
cdc_mask_tweets_week_5 = cdc_mask[cdc_mask['week_number'] == 5].reset_index()
cdc_mask_tweets_week_6 = cdc_mask[cdc_mask['week_number'] == 6].reset_index()
cdc_mask_tweets_week_7 = cdc_mask[cdc_mask['week_number'] == 7].reset_index()
cdc_mask_tweets_week_8 = cdc_mask[cdc_mask['week_number'] == 8].reset_index()
cdc_mask_tweets_week_9 = cdc_mask[cdc_mask['week_number'] == 9].reset_index()
cdc_mask_tweets_week_10 = cdc_mask[cdc_mask['week_number'] == 10].reset_index()
cdc_mask_tweets_week_11 = cdc_mask[cdc_mask['week_number'] == 11].reset_index()
cdc_mask_tweets_week_12 = cdc_mask[cdc_mask['week_number'] == 12].reset_index()
cdc_mask_tweets_week_13 = cdc_mask[cdc_mask['week_number'] == 13].reset_index()
cdc_mask_tweets_week_14 = cdc_mask[cdc_mask['week_number'] == 14].reset_index()
cdc_mask_tweets_week_15 = cdc_mask[cdc_mask['week_number'] == 15].reset_index()
cdc_mask_tweets_week_16 = cdc_mask[cdc_mask['week_number'] == 16].reset_index()

In [32]:
def agg_time_list(df):
    time_list = []
    for i in range(df.shape[0]):  
        tweet_wordstring = df.loc[i,'text_tokens']
        tweet_wordstring_ascii = re.sub("([^\x00-\x7F])+"," ",tweet_wordstring) 
        # based on this: https://datascience.stackexchange.com/questions/46705/to-remove-chinese-characters-as-features
        tweet_wordlist = tweet_wordstring_ascii.split()
        time_list += tweet_wordlist
    
    return(time_list)

In [33]:
term_list_week_1 = agg_time_list(cdc_mask_tweets_week_1)
term_list_week_2 = agg_time_list(cdc_mask_tweets_week_2)
term_list_week_3 = agg_time_list(cdc_mask_tweets_week_3)
term_list_week_4 = agg_time_list(cdc_mask_tweets_week_4)
term_list_week_5 = agg_time_list(cdc_mask_tweets_week_5)
term_list_week_6 = agg_time_list(cdc_mask_tweets_week_6)
term_list_week_7 = agg_time_list(cdc_mask_tweets_week_7)
term_list_week_8 = agg_time_list(cdc_mask_tweets_week_8)
term_list_week_9 = agg_time_list(cdc_mask_tweets_week_9)
term_list_week_10 = agg_time_list(cdc_mask_tweets_week_10)
term_list_week_11 = agg_time_list(cdc_mask_tweets_week_11)
term_list_week_12 = agg_time_list(cdc_mask_tweets_week_12)
term_list_week_13 = agg_time_list(cdc_mask_tweets_week_13)
term_list_week_14 = agg_time_list(cdc_mask_tweets_week_14)
term_list_week_15 = agg_time_list(cdc_mask_tweets_week_15)
term_list_week_16 = agg_time_list(cdc_mask_tweets_week_16)

In [34]:
term_list_month_1 = agg_time_list(cdc_mask_tweets_month_1)
term_list_month_2 = agg_time_list(cdc_mask_tweets_month_2)
term_list_month_3 = agg_time_list(cdc_mask_tweets_month_3)
term_list_month_4 = agg_time_list(cdc_mask_tweets_month_4)
term_list_month_5 = agg_time_list(cdc_mask_tweets_month_5)

In [35]:
print('Total number of words - Week 1:', len(term_list_week_1))
print('Total number of words - Week 2:', len(term_list_week_2))
print('Total number of words - Week 3:', len(term_list_week_3))
print('Total number of words - Week 4:', len(term_list_week_4))
print('Total number of words - Week 5:', len(term_list_week_5))
print('Total number of words - Week 6:', len(term_list_week_6))
print('Total number of words - Week 7:', len(term_list_week_7))
print('Total number of words - Week 8:', len(term_list_week_8))
print('Total number of words - Week 9:', len(term_list_week_9))
print('Total number of words - Week 10:', len(term_list_week_10))
print('Total number of words - Week 11:', len(term_list_week_11))
print('Total number of words - Week 12:', len(term_list_week_12))
print('Total number of words - Week 13:', len(term_list_week_13))
print('Total number of words - Week 14:', len(term_list_week_14))
print('Total number of words - Week 15:', len(term_list_week_15))
print('Total number of words - Week 16:', len(term_list_week_16))

Total number of words - Week 1: 9374
Total number of words - Week 2: 21327
Total number of words - Week 3: 17093
Total number of words - Week 4: 18319
Total number of words - Week 5: 15909
Total number of words - Week 6: 117974
Total number of words - Week 7: 52845
Total number of words - Week 8: 6895
Total number of words - Week 9: 18819
Total number of words - Week 10: 35581
Total number of words - Week 11: 111600
Total number of words - Week 12: 68062
Total number of words - Week 13: 31084
Total number of words - Week 14: 14878
Total number of words - Week 15: 38596
Total number of words - Week 16: 19283


In [36]:
print('Total number of words - Month 1:', len(term_list_month_1))
print('Total number of words - Month 2:', len(term_list_month_2))
print('Total number of words - Month 3:', len(term_list_month_3))
print('Total number of words - Month 4:', len(term_list_month_4))
print('Total number of words - Month 5:', len(term_list_month_5))

Total number of words - Month 1: 29012
Total number of words - Month 2: 153074
Total number of words - Month 3: 149747
Total number of words - Month 4: 234342
Total number of words - Month 5: 31464


In [37]:
print('Total number of unique words - Week 1:', len(set(term_list_week_1)))
print('Total number of unique words - Week 2:', len(set(term_list_week_2)))
print('Total number of unique words - Week 3:', len(set(term_list_week_3)))
print('Total number of unique words - Week 4:', len(set(term_list_week_4)))
print('Total number of unique words - Week 5:', len(set(term_list_week_5)))
print('Total number of unique words - Week 6:', len(set(term_list_week_6)))
print('Total number of unique words - Week 7:', len(set(term_list_week_7)))
print('Total number of unique words - Week 8:', len(set(term_list_week_8)))
print('Total number of unique words - Week 9:', len(set(term_list_week_9)))
print('Total number of unique words - Week 10:', len(set(term_list_week_10)))
print('Total number of unique words - Week 11:', len(set(term_list_week_11)))
print('Total number of unique words - Week 12:', len(set(term_list_week_12)))
print('Total number of unique words - Week 13:', len(set(term_list_week_13)))
print('Total number of unique words - Week 13:', len(set(term_list_week_14)))
print('Total number of unique words - Week 13:', len(set(term_list_week_15)))
print('Total number of unique words - Week 13:', len(set(term_list_week_16)))

Total number of unique words - Week 1: 1658
Total number of unique words - Week 2: 2665
Total number of unique words - Week 3: 1887
Total number of unique words - Week 4: 1934
Total number of unique words - Week 5: 1785
Total number of unique words - Week 6: 5713
Total number of unique words - Week 7: 4388
Total number of unique words - Week 8: 1725
Total number of unique words - Week 9: 2691
Total number of unique words - Week 10: 3057
Total number of unique words - Week 11: 5934
Total number of unique words - Week 12: 3950
Total number of unique words - Week 13: 3123
Total number of unique words - Week 13: 2836
Total number of unique words - Week 13: 3607
Total number of unique words - Week 13: 3226


In [38]:
print('Total number of unique words - Month 1:', len(set(term_list_month_1)))
print('Total number of unique words - Month 2:', len(set(term_list_month_2)))
print('Total number of unique words - Month 3:', len(set(term_list_month_3)))
print('Total number of unique words - Month 4:', len(set(term_list_month_4)))
print('Total number of unique words - Month 5:', len(set(term_list_month_5)))

Total number of unique words - Month 1: 3192
Total number of unique words - Month 2: 6659
Total number of unique words - Month 3: 7966
Total number of unique words - Month 4: 9056
Total number of unique words - Month 5: 4195


In [39]:
import numpy as np

print('% of unique words out of total word volume - Week 1:', np.floor(100*len(set(term_list_week_1))/len(term_list_week_1)))
print('% of unique words out of total word volume - Week 2:', np.floor(100*len(set(term_list_week_2))/len(term_list_week_2)))
print('% of unique words out of total word volume - Week 3:', np.floor(100*len(set(term_list_week_3))/len(term_list_week_3)))
print('% of unique words out of total word volume - Week 4:', np.floor(100*len(set(term_list_week_4))/len(term_list_week_4)))
print('% of unique words out of total word volume - Week 5:', np.floor(100*len(set(term_list_week_5))/len(term_list_week_5)))
print('% of unique words out of total word volume - Week 6:', np.floor(100*len(set(term_list_week_6))/len(term_list_week_6)))
print('% of unique words out of total word volume - Week 7:', np.floor(100*len(set(term_list_week_7))/len(term_list_week_7)))
print('% of unique words out of total word volume - Week 8:', np.floor(100*len(set(term_list_week_8))/len(term_list_week_8)))
print('% of unique words out of total word volume - Week 9:', np.floor(100*len(set(term_list_week_9))/len(term_list_week_9)))
print('% of unique words out of total word volume - Week 10:', np.floor(100*len(set(term_list_week_10))/len(term_list_week_10)))
print('% of unique words out of total word volume - Week 11:', np.floor(100*len(set(term_list_week_11))/len(term_list_week_11)))
print('% of unique words out of total word volume - Week 12:', np.floor(100*len(set(term_list_week_12))/len(term_list_week_12)))
print('% of unique words out of total word volume - Week 13:', np.floor(100*len(set(term_list_week_13))/len(term_list_week_13)))
print('% of unique words out of total word volume - Week 14:', np.floor(100*len(set(term_list_week_14))/len(term_list_week_14)))
print('% of unique words out of total word volume - Week 15:', np.floor(100*len(set(term_list_week_15))/len(term_list_week_15)))
print('% of unique words out of total word volume - Week 16:', np.floor(100*len(set(term_list_week_16))/len(term_list_week_16)))

% of unique words out of total word volume - Week 1: 17.0
% of unique words out of total word volume - Week 2: 12.0
% of unique words out of total word volume - Week 3: 11.0
% of unique words out of total word volume - Week 4: 10.0
% of unique words out of total word volume - Week 5: 11.0
% of unique words out of total word volume - Week 6: 4.0
% of unique words out of total word volume - Week 7: 8.0
% of unique words out of total word volume - Week 8: 25.0
% of unique words out of total word volume - Week 9: 14.0
% of unique words out of total word volume - Week 10: 8.0
% of unique words out of total word volume - Week 11: 5.0
% of unique words out of total word volume - Week 12: 5.0
% of unique words out of total word volume - Week 13: 10.0
% of unique words out of total word volume - Week 14: 19.0
% of unique words out of total word volume - Week 15: 9.0
% of unique words out of total word volume - Week 16: 16.0


In [40]:
print('% of unique words out of total word volume - Month 1:', np.floor(100*len(set(term_list_month_1))/len(term_list_month_1)))
print('% of unique words out of total word volume - Month 2:', np.floor(100*len(set(term_list_month_2))/len(term_list_month_2)))
print('% of unique words out of total word volume - Month 3:', np.floor(100*len(set(term_list_month_3))/len(term_list_month_3)))
print('% of unique words out of total word volume - Month 4:', np.floor(100*len(set(term_list_month_4))/len(term_list_month_4)))
print('% of unique words out of total word volume - Month 5:', np.floor(100*len(set(term_list_month_5))/len(term_list_month_5)))

% of unique words out of total word volume - Month 1: 11.0
% of unique words out of total word volume - Month 2: 4.0
% of unique words out of total word volume - Month 3: 5.0
% of unique words out of total word volume - Month 4: 3.0
% of unique words out of total word volume - Month 5: 13.0


In [41]:
# Given a list of words, return a dictionary of
# word-frequency pairs.

def word_list_to_freq_dict(wordlist):
    word_freq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,word_freq)))

In [42]:
# Sort a dictionary of word-frequency pairs in
# order of descending frequency.

def sort_freq_dict(freq_dict):
    aux = [(freq_dict[key], key) for key in freq_dict]
    aux.sort()
    aux.reverse()
    return aux

In [43]:
freq_dict_week_1 = word_list_to_freq_dict(term_list_week_1)
freq_dict_week_2 = word_list_to_freq_dict(term_list_week_2)
freq_dict_week_3 = word_list_to_freq_dict(term_list_week_3)
freq_dict_week_4 = word_list_to_freq_dict(term_list_week_4)
freq_dict_week_5 = word_list_to_freq_dict(term_list_week_5)
freq_dict_week_6 = word_list_to_freq_dict(term_list_week_6)
freq_dict_week_7 = word_list_to_freq_dict(term_list_week_7)
freq_dict_week_8 = word_list_to_freq_dict(term_list_week_8)
freq_dict_week_9 = word_list_to_freq_dict(term_list_week_9)
freq_dict_week_10 = word_list_to_freq_dict(term_list_week_10)
freq_dict_week_11 = word_list_to_freq_dict(term_list_week_11)
freq_dict_week_12 = word_list_to_freq_dict(term_list_week_12)
freq_dict_week_13 = word_list_to_freq_dict(term_list_week_13)
freq_dict_week_14 = word_list_to_freq_dict(term_list_week_14)
freq_dict_week_15 = word_list_to_freq_dict(term_list_week_15)
freq_dict_week_16 = word_list_to_freq_dict(term_list_week_16)

In [44]:
freq_dict_month_1 = word_list_to_freq_dict(term_list_month_1)
freq_dict_month_2 = word_list_to_freq_dict(term_list_month_2)
freq_dict_month_3 = word_list_to_freq_dict(term_list_month_3)
freq_dict_month_4 = word_list_to_freq_dict(term_list_month_4)
freq_dict_month_5 = word_list_to_freq_dict(term_list_month_5)

In [45]:
sorted_freq_dict_week_1 = sort_freq_dict(freq_dict_week_1)
sorted_freq_dict_week_2 = sort_freq_dict(freq_dict_week_2)
sorted_freq_dict_week_3 = sort_freq_dict(freq_dict_week_3)
sorted_freq_dict_week_4 = sort_freq_dict(freq_dict_week_4)
sorted_freq_dict_week_5 = sort_freq_dict(freq_dict_week_5)
sorted_freq_dict_week_6 = sort_freq_dict(freq_dict_week_6)
sorted_freq_dict_week_7 = sort_freq_dict(freq_dict_week_7)
sorted_freq_dict_week_8 = sort_freq_dict(freq_dict_week_8)
sorted_freq_dict_week_9 = sort_freq_dict(freq_dict_week_9)
sorted_freq_dict_week_10 = sort_freq_dict(freq_dict_week_10)
sorted_freq_dict_week_11 = sort_freq_dict(freq_dict_week_11)
sorted_freq_dict_week_12 = sort_freq_dict(freq_dict_week_12)
sorted_freq_dict_week_13 = sort_freq_dict(freq_dict_week_13)
sorted_freq_dict_week_14 = sort_freq_dict(freq_dict_week_14)
sorted_freq_dict_week_15 = sort_freq_dict(freq_dict_week_15)
sorted_freq_dict_week_16 = sort_freq_dict(freq_dict_week_16)

In [46]:
sorted_freq_dict_month_1 = sort_freq_dict(freq_dict_month_1)
sorted_freq_dict_month_2 = sort_freq_dict(freq_dict_month_2)
sorted_freq_dict_month_3 = sort_freq_dict(freq_dict_month_3)
sorted_freq_dict_month_4 = sort_freq_dict(freq_dict_month_4)
sorted_freq_dict_month_5 = sort_freq_dict(freq_dict_month_5)

In [47]:
sorted_freq_dict_week_1_df = pd.DataFrame(sorted_freq_dict_week_1, columns =['Frequency_Week_1', 'Word']) 
sorted_freq_dict_week_2_df = pd.DataFrame(sorted_freq_dict_week_2, columns =['Frequency_Week_2', 'Word']) 
sorted_freq_dict_week_3_df = pd.DataFrame(sorted_freq_dict_week_3, columns =['Frequency_Week_3', 'Word']) 
sorted_freq_dict_week_4_df = pd.DataFrame(sorted_freq_dict_week_4, columns =['Frequency_Week_4', 'Word'])
sorted_freq_dict_week_5_df = pd.DataFrame(sorted_freq_dict_week_5, columns =['Frequency_Week_5', 'Word']) 
sorted_freq_dict_week_6_df = pd.DataFrame(sorted_freq_dict_week_6, columns =['Frequency_Week_6', 'Word']) 
sorted_freq_dict_week_7_df = pd.DataFrame(sorted_freq_dict_week_7, columns =['Frequency_Week_7', 'Word']) 
sorted_freq_dict_week_8_df = pd.DataFrame(sorted_freq_dict_week_8, columns =['Frequency_Week_8', 'Word']) 
sorted_freq_dict_week_9_df = pd.DataFrame(sorted_freq_dict_week_9, columns =['Frequency_Week_9', 'Word']) 
sorted_freq_dict_week_10_df = pd.DataFrame(sorted_freq_dict_week_10, columns =['Frequency_Week_10', 'Word']) 
sorted_freq_dict_week_11_df = pd.DataFrame(sorted_freq_dict_week_11, columns =['Frequency_Week_11', 'Word']) 
sorted_freq_dict_week_12_df = pd.DataFrame(sorted_freq_dict_week_12, columns =['Frequency_Week_12', 'Word'])
sorted_freq_dict_week_13_df = pd.DataFrame(sorted_freq_dict_week_13, columns =['Frequency_Week_13', 'Word']) 
sorted_freq_dict_week_14_df = pd.DataFrame(sorted_freq_dict_week_14, columns =['Frequency_Week_14', 'Word']) 
sorted_freq_dict_week_15_df = pd.DataFrame(sorted_freq_dict_week_15, columns =['Frequency_Week_15', 'Word']) 
sorted_freq_dict_week_16_df = pd.DataFrame(sorted_freq_dict_week_16, columns =['Frequency_Week_16', 'Word']) 

In [48]:
sorted_freq_dict_week_1_df['Rank_Week_1'] = [i+1 for i in range(sorted_freq_dict_week_1_df.shape[0])]
sorted_freq_dict_week_2_df['Rank_Week_2'] = [i+1 for i in range(sorted_freq_dict_week_2_df.shape[0])]
sorted_freq_dict_week_3_df['Rank_Week_3'] = [i+1 for i in range(sorted_freq_dict_week_3_df.shape[0])]
sorted_freq_dict_week_4_df['Rank_Week_4'] = [i+1 for i in range(sorted_freq_dict_week_4_df.shape[0])]
sorted_freq_dict_week_5_df['Rank_Week_5'] = [i+1 for i in range(sorted_freq_dict_week_5_df.shape[0])]
sorted_freq_dict_week_6_df['Rank_Week_6'] = [i+1 for i in range(sorted_freq_dict_week_6_df.shape[0])]
sorted_freq_dict_week_7_df['Rank_Week_7'] = [i+1 for i in range(sorted_freq_dict_week_7_df.shape[0])]
sorted_freq_dict_week_8_df['Rank_Week_8'] = [i+1 for i in range(sorted_freq_dict_week_8_df.shape[0])]
sorted_freq_dict_week_9_df['Rank_Week_9'] = [i+1 for i in range(sorted_freq_dict_week_9_df.shape[0])]
sorted_freq_dict_week_10_df['Rank_Week_10'] = [i+1 for i in range(sorted_freq_dict_week_10_df.shape[0])]
sorted_freq_dict_week_11_df['Rank_Week_11'] = [i+1 for i in range(sorted_freq_dict_week_11_df.shape[0])]
sorted_freq_dict_week_12_df['Rank_Week_12'] = [i+1 for i in range(sorted_freq_dict_week_12_df.shape[0])]
sorted_freq_dict_week_13_df['Rank_Week_13'] = [i+1 for i in range(sorted_freq_dict_week_13_df.shape[0])]
sorted_freq_dict_week_14_df['Rank_Week_14'] = [i+1 for i in range(sorted_freq_dict_week_14_df.shape[0])]
sorted_freq_dict_week_15_df['Rank_Week_15'] = [i+1 for i in range(sorted_freq_dict_week_15_df.shape[0])]
sorted_freq_dict_week_16_df['Rank_Week_16'] = [i+1 for i in range(sorted_freq_dict_week_16_df.shape[0])]

In [49]:
sorted_freq_dict_month_1_df = pd.DataFrame(sorted_freq_dict_month_1, columns =['Frequency_Month_1', 'Word']) 
sorted_freq_dict_month_2_df = pd.DataFrame(sorted_freq_dict_month_2, columns =['Frequency_Month_2', 'Word']) 
sorted_freq_dict_month_3_df = pd.DataFrame(sorted_freq_dict_month_3, columns =['Frequency_Month_3', 'Word']) 
sorted_freq_dict_month_4_df = pd.DataFrame(sorted_freq_dict_month_4, columns =['Frequency_Month_4', 'Word'])
sorted_freq_dict_month_5_df = pd.DataFrame(sorted_freq_dict_month_5, columns =['Frequency_Month_5', 'Word']) 

In [50]:
sorted_freq_dict_month_1_df['Rank_Month_1'] = [i+1 for i in range(sorted_freq_dict_month_1_df.shape[0])]
sorted_freq_dict_month_2_df['Rank_Month_2'] = [i+1 for i in range(sorted_freq_dict_month_2_df.shape[0])]
sorted_freq_dict_month_3_df['Rank_Month_3'] = [i+1 for i in range(sorted_freq_dict_month_3_df.shape[0])]
sorted_freq_dict_month_4_df['Rank_Month_4'] = [i+1 for i in range(sorted_freq_dict_month_4_df.shape[0])]
sorted_freq_dict_month_5_df['Rank_Month_5'] = [i+1 for i in range(sorted_freq_dict_month_5_df.shape[0])]

In [51]:
sorted_freq_dict_week_1_df.head(10)

,Frequency_Week_1,Word,Rank_Week_1
0,356,facemask,1
1,327,protect,2
2,324,coronavirus,3
3,319,say,4
4,307,wont,5
5,303,likely,6
6,302,coronovirus,7
7,302,contracting,8
8,276,question,9
9,119,wearing,10


In [52]:
sorted_freq_dict_week_2_df.head(10)

,Frequency_Week_2,Word,Rank_Week_2
0,835,case,1
1,713,say,2
2,679,public,3
3,546,person,4
4,545,facemask,5
5,543,coronavirus,6
6,457,wear,7
7,354,recommending,8
8,329,ha,9
9,319,health,10


In [53]:
sorted_freq_dict_week_3_df.head(10)

,Frequency_Week_3,Word,Rank_Week_3
0,948,help,1
1,633,facemask,2
2,576,use,3
3,531,recommend,4
4,525,doe,5
5,520,spread,6
6,514,take,7
7,512,prevent,8
8,506,currently,9
9,476,spreading,10


In [54]:
sorted_freq_dict_week_4_df.head(10)

,Frequency_Week_4,Word,Rank_Week_4
0,1100,facemask,1
1,1005,also,2
2,993,wearing,3
3,987,coronavirus,4
4,982,case,5
5,967,recommends,6
6,954,back,7
7,953,head,8
8,952,rear,9
9,952,attack,10


In [55]:
sorted_freq_dict_week_5_df.head(10)

,Frequency_Week_5,Word,Rank_Week_5
0,801,case,1
1,774,facemask,2
2,652,wearing,3
3,642,coronavirus,4
4,625,also,5
5,623,back,6
6,618,recommends,7
7,617,head,8
8,616,rear,9
9,616,attack,10


In [56]:
sorted_freq_dict_week_6_df.head(10)

,Frequency_Week_6,Word,Rank_Week_6
0,3481,facemask,1
1,2697,help,2
2,2454,recommend,3
3,2363,doe,4
4,2007,respiratory,5
5,1773,wear,6
6,1747,spread,7
7,1730,use,8
8,1677,sick,9
9,1610,hand,10


In [57]:
sorted_freq_dict_week_7_df.head(10)

,Frequency_Week_7,Word,Rank_Week_7
0,1260,help,1
1,1208,facemask,2
2,918,recommend,3
3,900,hand,4
4,860,sick,5
5,851,doe,6
6,768,use,7
7,742,home,8
8,731,people,9
9,713,spread,10


In [58]:
sorted_freq_dict_week_8_df.head(10)

,Frequency_Week_8,Word,Rank_Week_8
0,240,people,1
1,209,hand,2
2,142,wash,3
3,124,need,4
4,97,facemask,5
5,95,wear,6
6,87,omg,7
7,70,coronavirus,8
8,56,help,9
9,50,chinese,10


In [59]:
sorted_freq_dict_week_9_df.head(10)

,Frequency_Week_9,Word,Rank_Week_9
0,473,use,1
1,272,health,2
2,269,facemask,3
3,247,care,4
4,225,worker,5
5,200,make,6
6,198,wear,7
7,197,shortage,8
8,187,covid19,9
9,186,say,10


In [60]:
sorted_freq_dict_week_10_df.head(10)

,Frequency_Week_10,Word,Rank_Week_10
0,996,wear,1
1,943,guidance,2
2,743,new,3
3,723,american,4
4,714,current,5
5,702,recommendation,6
6,699,life,7
7,686,change,8
8,677,day,9
9,664,everyday,10


In [61]:
sorted_freq_dict_week_11_df.head(10)

,Frequency_Week_11,Word,Rank_Week_11
0,3460,wear,1
1,3288,face,2
2,3124,covering,3
3,2546,public,4
4,2401,cloth,5
5,1866,wearing,6
6,1764,recommendation,7
7,1395,make,8
8,1337,trump,9
9,1219,say,10


In [62]:
sorted_freq_dict_week_12_df.head(10)

,Frequency_Week_12,Word,Rank_Week_12
0,2788,wear,1
1,1983,social,2
2,1971,distancing,3
3,1734,face,4
4,1694,covering,5
5,1660,people,6
6,1608,public,7
7,1579,spread,8
8,1550,cloth,9
9,1077,setting,10


In [63]:
sorted_freq_dict_week_13_df.head(10)

,Frequency_Week_13,Word,Rank_Week_13
0,622,wear,1
1,610,facemask,2
2,492,house,3
3,490,white,4
4,379,face,5
5,358,know,6
6,358,china,7
7,357,covering,8
8,350,trump,9
9,333,public,10


In [64]:
sorted_freq_dict_week_14_df.head(10)

,Frequency_Week_14,Word,Rank_Week_14
0,353,wear,1
1,204,cloth,2
2,195,facemask,3
3,190,wearing,4
4,161,face,5
5,135,public,6
6,135,covering,7
7,129,amp,8
8,122,people,9
9,115,use,10


In [65]:
sorted_freq_dict_week_15_df.head(10)

,Frequency_Week_15,Word,Rank_Week_15
0,1150,wear,1
1,893,sure,2
2,868,slowly,3
3,741,face,4
4,617,guideline,5
5,579,amp,6
6,548,social,7
7,546,virus,8
8,546,distancing,9
9,534,make,10


In [66]:
sorted_freq_dict_week_16_df.head(10)

,Frequency_Week_16,Word,Rank_Week_16
0,529,wear,1
1,236,wearing,2
2,235,people,3
3,203,face,4
4,201,facemask,5
5,193,amp,6
6,172,covering,7
7,144,need,8
8,140,cloth,9
9,136,protect,10


In [111]:
sorted_freq_dict_month_1_df.head(10)

,Frequency_Month_1,Word,Rank_Month_1
0,1001,say,1
1,867,case,2
2,858,facemask,3
3,823,coronavirus,4
4,711,public,5
5,549,person,6
6,519,wear,7
7,380,protect,8
8,349,health,9
9,340,recommending,10


In [112]:
sorted_freq_dict_month_2_df.head(10)

,Frequency_Month_2,Word,Rank_Month_2
0,5611,facemask,1
1,3414,help,2
2,2800,recommend,3
3,2752,doe,4
4,2604,coronavirus,5
5,2350,respiratory,6
6,2258,use,7
7,2194,wearing,8
8,2095,spread,9
9,1919,case,10


In [113]:
sorted_freq_dict_month_3_df.head(10)

,Frequency_Month_3,Word,Rank_Month_3
0,3038,wear,1
1,2308,facemask,2
2,1832,use,3
3,1713,help,4
4,1693,people,5
5,1417,hand,6
6,1381,recommend,7
7,1346,everyday,8
8,1317,public,9
9,1289,like,10


In [114]:
sorted_freq_dict_month_4_df.head(10)

,Frequency_Month_4,Word,Rank_Month_4
0,7252,wear,1
1,5794,face,2
2,5109,covering,3
3,4580,cloth,4
4,3992,public,5
5,3089,people,6
6,3033,wearing,7
7,2930,social,8
8,2863,distancing,9
9,2745,make,10


In [115]:
sorted_freq_dict_month_5_df.head(10)

,Frequency_Month_5,Word,Rank_Month_5
0,909,wear,1
1,350,wearing,2
2,350,people,3
3,337,facemask,4
4,315,death,5
5,308,face,6
6,258,covering,7
7,256,amp,8
8,252,done,9
9,226,use,10


In [67]:
print('Number of unique words appearing at least once - Week 1: ', sorted_freq_dict_week_1_df.shape[0])
print('Number of unique words appearing at least once - Week 2: ', sorted_freq_dict_week_2_df.shape[0])
print('Number of unique words appearing at least once - Week 3: ', sorted_freq_dict_week_3_df.shape[0])
print('Number of unique words appearing at least once - Week 4: ', sorted_freq_dict_week_4_df.shape[0])
print('Number of unique words appearing at least once - Week 5: ', sorted_freq_dict_week_5_df.shape[0])
print('Number of unique words appearing at least once - Week 6: ', sorted_freq_dict_week_6_df.shape[0])
print('Number of unique words appearing at least once - Week 7: ', sorted_freq_dict_week_7_df.shape[0])
print('Number of unique words appearing at least once - Week 8: ', sorted_freq_dict_week_8_df.shape[0])
print('Number of unique words appearing at least once - Week 9: ', sorted_freq_dict_week_9_df.shape[0])
print('Number of unique words appearing at least once - Week 10: ', sorted_freq_dict_week_10_df.shape[0])
print('Number of unique words appearing at least once - Week 11: ', sorted_freq_dict_week_11_df.shape[0])
print('Number of unique words appearing at least once - Week 12: ', sorted_freq_dict_week_12_df.shape[0])
print('Number of unique words appearing at least once - Week 13: ', sorted_freq_dict_week_13_df.shape[0])
print('Number of unique words appearing at least once - Week 14: ', sorted_freq_dict_week_14_df.shape[0])
print('Number of unique words appearing at least once - Week 15: ', sorted_freq_dict_week_15_df.shape[0])
print('Number of unique words appearing at least once - Week 16: ', sorted_freq_dict_week_16_df.shape[0])

Number of unique words appearing at least once - Week 1:  1658
Number of unique words appearing at least once - Week 2:  2665
Number of unique words appearing at least once - Week 3:  1887
Number of unique words appearing at least once - Week 4:  1934
Number of unique words appearing at least once - Week 5:  1785
Number of unique words appearing at least once - Week 6:  5713
Number of unique words appearing at least once - Week 7:  4388
Number of unique words appearing at least once - Week 8:  1725
Number of unique words appearing at least once - Week 9:  2691
Number of unique words appearing at least once - Week 10:  3057
Number of unique words appearing at least once - Week 11:  5934
Number of unique words appearing at least once - Week 12:  3950
Number of unique words appearing at least once - Week 13:  3123
Number of unique words appearing at least once - Week 14:  2836
Number of unique words appearing at least once - Week 15:  3607
Number of unique words appearing at least once - 

In [69]:
print('Number of unique words appearing at least twice - Week 1: ', sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency_Week_1'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 2: ', sorted_freq_dict_week_2_df[sorted_freq_dict_week_2_df['Frequency_Week_2'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 3: ', sorted_freq_dict_week_3_df[sorted_freq_dict_week_3_df['Frequency_Week_3'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 4: ', sorted_freq_dict_week_4_df[sorted_freq_dict_week_4_df['Frequency_Week_4'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 5: ', sorted_freq_dict_week_5_df[sorted_freq_dict_week_5_df['Frequency_Week_5'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 6: ', sorted_freq_dict_week_6_df[sorted_freq_dict_week_6_df['Frequency_Week_6'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 7: ', sorted_freq_dict_week_7_df[sorted_freq_dict_week_7_df['Frequency_Week_7'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 8: ', sorted_freq_dict_week_8_df[sorted_freq_dict_week_8_df['Frequency_Week_8'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 9: ', sorted_freq_dict_week_9_df[sorted_freq_dict_week_9_df['Frequency_Week_9'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 10: ', sorted_freq_dict_week_10_df[sorted_freq_dict_week_10_df['Frequency_Week_10'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 11: ', sorted_freq_dict_week_11_df[sorted_freq_dict_week_11_df['Frequency_Week_11'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 12: ', sorted_freq_dict_week_12_df[sorted_freq_dict_week_12_df['Frequency_Week_12'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 13: ', sorted_freq_dict_week_13_df[sorted_freq_dict_week_13_df['Frequency_Week_13'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 14: ', sorted_freq_dict_week_14_df[sorted_freq_dict_week_14_df['Frequency_Week_14'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 15: ', sorted_freq_dict_week_15_df[sorted_freq_dict_week_15_df['Frequency_Week_15'] > 1].shape[0])
print('Number of unique words appearing at least twice - Week 16: ', sorted_freq_dict_week_16_df[sorted_freq_dict_week_16_df['Frequency_Week_16'] > 1].shape[0])

Number of unique words appearing at least twice - Week 1:  802
Number of unique words appearing at least twice - Week 2:  1284
Number of unique words appearing at least twice - Week 3:  928
Number of unique words appearing at least twice - Week 4:  910
Number of unique words appearing at least twice - Week 5:  824
Number of unique words appearing at least twice - Week 6:  3074
Number of unique words appearing at least twice - Week 7:  2267
Number of unique words appearing at least twice - Week 8:  782
Number of unique words appearing at least twice - Week 9:  1319
Number of unique words appearing at least twice - Week 10:  1567
Number of unique words appearing at least twice - Week 11:  3239
Number of unique words appearing at least twice - Week 12:  2007
Number of unique words appearing at least twice - Week 13:  1536
Number of unique words appearing at least twice - Week 14:  1379
Number of unique words appearing at least twice - Week 15:  1767
Number of unique words appearing at lea

In [ ]:
print('Number of unique words appearing at least 5 times - Week 1: ', sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 2: ', sorted_freq_dict_week_2_df[sorted_freq_dict_week_2_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 3: ', sorted_freq_dict_week_3_df[sorted_freq_dict_week_3_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 4: ', sorted_freq_dict_week_4_df[sorted_freq_dict_week_4_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 5: ', sorted_freq_dict_week_5_df[sorted_freq_dict_week_5_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 6: ', sorted_freq_dict_week_6_df[sorted_freq_dict_week_6_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 7: ', sorted_freq_dict_week_7_df[sorted_freq_dict_week_7_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 8: ', sorted_freq_dict_week_8_df[sorted_freq_dict_week_8_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 9: ', sorted_freq_dict_week_9_df[sorted_freq_dict_week_9_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 10: ', sorted_freq_dict_week_10_df[sorted_freq_dict_week_10_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 11: ', sorted_freq_dict_week_11_df[sorted_freq_dict_week_11_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 12: ', sorted_freq_dict_week_12_df[sorted_freq_dict_week_12_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 13: ', sorted_freq_dict_week_13_df[sorted_freq_dict_week_13_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 14: ', sorted_freq_dict_week_14_df[sorted_freq_dict_week_14_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 15: ', sorted_freq_dict_week_15_df[sorted_freq_dict_week_15_df['Frequency'] > 4].shape[0])
print('Number of unique words appearing at least 5 times - Week 16: ', sorted_freq_dict_week_16_df[sorted_freq_dict_week_16_df['Frequency'] > 4].shape[0])

In [ ]:
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape
sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Frequency'] > 9].shape

In [ ]:
# Words in common between weeks 1, 2, ..., 16

In [70]:
sorted_freq_dict_weeks_df = sorted_freq_dict_week_1_df.merge(sorted_freq_dict_week_2_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_3_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_4_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_5_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_6_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_7_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_8_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_9_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_10_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_11_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_12_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_13_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_14_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_15_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_df = sorted_freq_dict_weeks_df.merge(sorted_freq_dict_week_16_df, how='inner', left_on='Word', right_on='Word')

In [71]:
sorted_freq_dict_months_df = sorted_freq_dict_month_1_df.merge(sorted_freq_dict_month_2_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_months_df = sorted_freq_dict_months_df.merge(sorted_freq_dict_month_3_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_months_df = sorted_freq_dict_months_df.merge(sorted_freq_dict_month_4_df, how='inner', left_on='Word', right_on='Word')
sorted_freq_dict_months_df = sorted_freq_dict_months_df.merge(sorted_freq_dict_month_5_df, how='inner', left_on='Word', right_on='Word')

In [72]:
sorted_freq_dict_weeks_large_df = sorted_freq_dict_week_1_df.merge(sorted_freq_dict_week_2_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_3_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_4_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_5_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_6_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_7_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_8_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_9_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_10_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_11_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_12_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_13_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_14_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_15_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_weeks_large_df = sorted_freq_dict_weeks_large_df.merge(sorted_freq_dict_week_16_df, how='outer', left_on='Word', right_on='Word')

In [73]:
sorted_freq_dict_months_large_df = sorted_freq_dict_month_1_df.merge(sorted_freq_dict_month_2_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_months_large_df = sorted_freq_dict_months_large_df.merge(sorted_freq_dict_month_3_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_months_large_df = sorted_freq_dict_months_large_df.merge(sorted_freq_dict_month_4_df, how='outer', left_on='Word', right_on='Word')
sorted_freq_dict_months_large_df = sorted_freq_dict_months_large_df.merge(sorted_freq_dict_month_5_df, how='outer', left_on='Word', right_on='Word')

In [74]:
sorted_freq_dict_weeks_df.shape

(342, 33)

In [75]:
sorted_freq_dict_months_df.shape

(1461, 11)

In [76]:
sorted_freq_dict_weeks_large_df.shape

(16070, 33)

In [77]:
sorted_freq_dict_months_large_df.shape

(16070, 11)

In [82]:
sorted_freq_dict_weeks_df.head(25)

,Frequency_Week_1,Word,Rank_Week_1,Frequency_Week_2,Rank_Week_2,Frequency_Week_3,Rank_Week_3,Frequency_Week_4,Rank_Week_4,Frequency_Week_5,...,Frequency_Week_12,Rank_Week_12,Frequency_Week_13,Rank_Week_13,Frequency_Week_14,Rank_Week_14,Frequency_Week_15,Rank_Week_15,Frequency_Week_16,Rank_Week_16
0,356,facemask,1,545,5,633,2,1100,1,774,...,949,12,610,2,195,3,270,33,201,5
1,327,protect,2,59,46,41,35,34,53,15,...,208,50,216,35,83,19,106,61,136,10
2,324,coronavirus,3,543,6,115,19,987,4,642,...,619,24,308,12,53,44,45,119,39,83
3,319,say,4,713,2,82,24,45,30,78,...,162,66,50,117,88,15,136,45,85,30
4,307,wont,5,26,131,10,166,6,289,1,...,9,648,9,448,10,270,15,334,11,343
5,303,likely,6,37,83,5,347,4,451,6,...,6,850,4,814,4,698,3,1117,4,849
6,276,question,9,24,151,3,512,6,312,3,...,19,381,18,299,7,397,22,238,13,296
7,119,wearing,10,155,21,89,23,993,3,652,...,571,25,250,19,190,4,380,28,236,2
8,84,wear,11,457,7,102,21,81,15,64,...,2788,1,622,1,353,1,1150,1,529,1
9,82,recommends,12,52,57,15,110,967,6,618,...,422,36,165,37,83,18,139,42,119,15


In [127]:
sorted_freq_dict_months_df.head(25)

,Frequency_Month_1,Word,Rank_Month_1,Frequency_Month_2,Rank_Month_2,Frequency_Month_3,Rank_Month_3,Frequency_Month_4,Rank_Month_4,Frequency_Month_5,Rank_Month_5
0,1001,say,1,587,52,797,32,1507,21,157,27
1,867,case,2,1919,10,196,142,155,244,24,244
2,858,facemask,3,5611,1,2308,2,2684,11,337,4
3,823,coronavirus,4,2604,5,882,25,1644,15,52,106
4,711,public,5,750,43,1317,9,3992,5,225,11
5,549,person,6,114,201,193,145,223,183,35,155
6,519,wear,7,1732,15,3038,1,7252,1,909,1
7,380,protect,8,1071,37,746,37,1585,18,168,20
8,349,health,9,270,76,885,24,485,95,78,74
9,340,recommending,10,90,254,560,58,947,39,38,146


In [128]:
sorted_freq_dict_weeks_large_df.head(25)

,Frequency_Week_1,Word,Rank_Week_1,Frequency_Week_2,Rank_Week_2,Frequency_Week_3,Rank_Week_3,Frequency_Week_4,Rank_Week_4,Frequency_Week_5,...,Rank_Week_14,Frequency_Week_15,Rank_Week_15,Frequency_Week_16,Rank_Week_16,Frequency_Weeks_1_4,Frequency_Weeks_5_8,Frequency_Weeks_9_12,Frequency_Weeks_13_16,Frequency_Weeks_1_16
0,356.0,facemask,1.0,545.0,5.0,633.0,2.0,1100.0,1.0,774.0,...,3.0,270.0,33.0,201.0,5.0,2634.0,5560.0,2328.0,1276.0,11798.0
1,327.0,protect,2.0,59.0,46.0,41.0,35.0,34.0,53.0,15.0,...,19.0,106.0,61.0,136.0,10.0,461.0,1587.0,1361.0,541.0,3950.0
2,324.0,coronavirus,3.0,543.0,6.0,115.0,19.0,987.0,4.0,642.0,...,44.0,45.0,119.0,39.0,83.0,1969.0,2086.0,1505.0,445.0,6005.0
3,319.0,say,4.0,713.0,2.0,82.0,24.0,45.0,30.0,78.0,...,15.0,136.0,45.0,85.0,30.0,1159.0,879.0,1652.0,359.0,4049.0
4,307.0,wont,5.0,26.0,131.0,10.0,166.0,6.0,289.0,1.0,...,270.0,15.0,334.0,11.0,343.0,349.0,119.0,202.0,45.0,715.0
5,303.0,likely,6.0,37.0,83.0,5.0,347.0,4.0,451.0,6.0,...,698.0,3.0,1117.0,4.0,849.0,349.0,39.0,78.0,15.0,481.0
6,302.0,coronovirus,7.0,17.0,210.0,1.0,1685.0,1.0,1734.0,NaN,...,NaN,NaN,NaN,NaN,NaN,321.0,NaN,NaN,NaN,NaN
7,302.0,contracting,8.0,19.0,183.0,1.0,1693.0,NaN,NaN,1.0,...,NaN,2.0,1675.0,1.0,2908.0,NaN,NaN,NaN,NaN,NaN
8,276.0,question,9.0,24.0,151.0,3.0,512.0,6.0,312.0,3.0,...,397.0,22.0,238.0,13.0,296.0,309.0,49.0,131.0,60.0,549.0
9,119.0,wearing,10.0,155.0,21.0,89.0,23.0,993.0,3.0,652.0,...,4.0,380.0,28.0,236.0,2.0,1356.0,1638.0,2749.0,1056.0,6799.0


In [129]:
sorted_freq_dict_months_large_df.head(25)

,Frequency_Month_1,Word,Rank_Month_1,Frequency_Month_2,Rank_Month_2,Frequency_Month_3,Rank_Month_3,Frequency_Month_4,Rank_Month_4,Frequency_Month_5,Rank_Month_5
0,1001.0,say,1.0,587.0,52.0,797.0,32.0,1507.0,21.0,157.0,27.0
1,867.0,case,2.0,1919.0,10.0,196.0,142.0,155.0,244.0,24.0,244.0
2,858.0,facemask,3.0,5611.0,1.0,2308.0,2.0,2684.0,11.0,337.0,4.0
3,823.0,coronavirus,4.0,2604.0,5.0,882.0,25.0,1644.0,15.0,52.0,106.0
4,711.0,public,5.0,750.0,43.0,1317.0,9.0,3992.0,5.0,225.0,11.0
5,549.0,person,6.0,114.0,201.0,193.0,145.0,223.0,183.0,35.0,155.0
6,519.0,wear,7.0,1732.0,15.0,3038.0,1.0,7252.0,1.0,909.0,1.0
7,380.0,protect,8.0,1071.0,37.0,746.0,37.0,1585.0,18.0,168.0,20.0
8,349.0,health,9.0,270.0,76.0,885.0,24.0,485.0,95.0,78.0,74.0
9,340.0,recommending,10.0,90.0,254.0,560.0,58.0,947.0,39.0,38.0,146.0


In [78]:
sorted_freq_dict_weeks_df.to_csv('Weekly_Ranks_Intersection.csv', encoding='utf-8', index=False)

In [79]:
sorted_freq_dict_months_df.to_csv('Monthly_Ranks_Intersection.csv', encoding='utf-8', index=False)

In [80]:
sorted_freq_dict_weeks_large_df.to_csv('Weekly_Ranks_Union.csv', encoding='utf-8', index=False)

In [81]:
sorted_freq_dict_months_large_df.to_csv('Monthly_Ranks_Union.csv', encoding='utf-8', index=False)

In [ ]:
# Words that had frequency of at least 2 consistently, and in common

In [ ]:
# Words that had frequency of at least 5 consistently, and in common

In [ ]:
# Words that had frequency of at least 10 consistently, and in common

In [83]:
# Total frequency, and then overall rank (for 16 weeks)
sorted_freq_dict_weeks_large_df['Frequency_Weeks_1_4'] = sorted_freq_dict_weeks_large_df['Frequency_Week_1'] + sorted_freq_dict_weeks_large_df['Frequency_Week_2'] + sorted_freq_dict_weeks_large_df['Frequency_Week_3'] + sorted_freq_dict_weeks_large_df['Frequency_Week_4']
sorted_freq_dict_weeks_large_df['Frequency_Weeks_5_8'] = sorted_freq_dict_weeks_large_df['Frequency_Week_5'] + sorted_freq_dict_weeks_large_df['Frequency_Week_6'] + sorted_freq_dict_weeks_large_df['Frequency_Week_7'] + sorted_freq_dict_weeks_large_df['Frequency_Week_8']
sorted_freq_dict_weeks_large_df['Frequency_Weeks_9_12'] = sorted_freq_dict_weeks_large_df['Frequency_Week_9'] + sorted_freq_dict_weeks_large_df['Frequency_Week_10'] + sorted_freq_dict_weeks_large_df['Frequency_Week_11'] + sorted_freq_dict_weeks_large_df['Frequency_Week_12']
sorted_freq_dict_weeks_large_df['Frequency_Weeks_13_16'] = sorted_freq_dict_weeks_large_df['Frequency_Week_13'] + sorted_freq_dict_weeks_large_df['Frequency_Week_14'] + sorted_freq_dict_weeks_large_df['Frequency_Week_15'] + sorted_freq_dict_weeks_large_df['Frequency_Week_16']

In [84]:
sorted_freq_dict_weeks_large_df['Frequency_Weeks_1_16'] = sorted_freq_dict_weeks_large_df['Frequency_Weeks_1_4'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_5_8'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_9_12'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_13_16']

In [85]:
sorted_freq_dict_weeks_large_sub_df = sorted_freq_dict_weeks_large_df[['Word','Frequency_Weeks_1_16']]
sorted_freq_dict_weeks_large_sub_df = sorted_freq_dict_weeks_large_sub_df.sort_values(by=['Frequency_Weeks_1_16'])
sorted_freq_dict_weeks_large_sub_df['Rank_Weeks_1_16'] = [i+1 for i in range(sorted_freq_dict_weeks_large_sub_df.shape[0])]

In [ ]:
# Total frequency, and then overall rank (for 5 months)

In [ ]:
sorted_freq_dict_months_large_df['Frequency_Months_1_5'] = sorted_freq_dict_months_large_df['Frequency_Weeks_1_4'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_5_8'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_9_12'] + sorted_freq_dict_weeks_large_df['Frequency_Weeks_13_16']

In [86]:
# words in week 1, not in subsequent weeks
# words in week 2, not in subsequent weeks (not in prior week either or in prior weeks but with frequency of 1, < 5)
# words in week 3, not in subsequent weeks (not in prior weeks either or in prior weeks but with frequency of 1, < 5)
words_week_1 = set(term_list_week_1)
words_rest_not_1 = set(term_list_week_2).union(set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), 
                                               set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11),
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                               set(term_list_week_15), set(term_list_week_16))

words_week_2 = set(term_list_week_2)
words_rest_not_2 = set(term_list_week_1).union(set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), 
                                               set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_2 = set(term_list_week_1)
words_rest_after_2 = set(term_list_week_3).union(set(term_list_week_4), set(term_list_week_5), set(term_list_week_6),
                                                 set(term_list_week_7), set(term_list_week_8), set(term_list_week_9),
                                                 set(term_list_week_10), set(term_list_week_11), set(term_list_week_12),
                                                 set(term_list_week_13), set(term_list_week_14), set(term_list_week_15), 
                                                 set(term_list_week_16))

words_week_3 = set(term_list_week_3)
words_rest_not_3 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_4), set(term_list_week_5), 
                                               set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_3 = set(term_list_week_1).union(set(term_list_week_2))
words_rest_after_3 = set(term_list_week_4).union(set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), 
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                 set(term_list_week_11), set(term_list_week_12), set(term_list_week_13), 
                                                 set(term_list_week_14), set(term_list_week_15), set(term_list_week_16))

words_week_4 = set(term_list_week_4)
words_rest_not_4 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_5), 
                                               set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_4 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3))
words_rest_after_4 = set(term_list_week_5).union(set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                                 set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

words_week_5 = set(term_list_week_5)
words_rest_not_5 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                               set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_5 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4))
words_rest_after_5 = set(term_list_week_6).union(set(term_list_week_7), set(term_list_week_8),
                                                 set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 6
words_week_6 = set(term_list_week_6)
words_rest_not_6 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                               set(term_list_week_5), set(term_list_week_7), set(term_list_week_8), 
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_6 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4),
                                                 set(term_list_week_5))
words_rest_after_6 = set(term_list_week_7).union(set(term_list_week_8),
                                                 set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 7
words_week_7 = set(term_list_week_7)
words_rest_not_7 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                               set(term_list_week_5), set(term_list_week_6), set(term_list_week_8), 
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_7 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6))
words_rest_after_7 = set(term_list_week_8).union(set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 8
words_week_8 = set(term_list_week_8)
words_rest_not_8 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                               set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                               set(term_list_week_9), set(term_list_week_10), set(term_list_week_11),
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_8 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7))
words_rest_after_8 = set(term_list_week_9).union(set(term_list_week_10), set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 9
words_week_9 = set(term_list_week_9)
words_rest_not_9 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                               set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), 
                                               set(term_list_week_8), set(term_list_week_10), set(term_list_week_11), 
                                               set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                               set(term_list_week_15), set(term_list_week_16))
words_rest_prior_9 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8))
words_rest_after_9 = set(term_list_week_10).union(set(term_list_week_11), 
                                                 set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 10
words_week_10 = set(term_list_week_10)
words_rest_not_10 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_11), set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                    set(term_list_week_15), set(term_list_week_16))
words_rest_prior_10 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9))
words_rest_after_10 = set(term_list_week_11).union(set(term_list_week_12), set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 11
words_week_11 = set(term_list_week_11)
words_rest_not_11 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), 
                                                set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                                                set(term_list_week_15), set(term_list_week_16))
words_rest_prior_11 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10))
words_rest_after_11 = set(term_list_week_12).union(set(term_list_week_13), set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 12
words_week_12 = set(term_list_week_12)
words_rest_not_12 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), set(term_list_week_13), set(term_list_week_14),
                    set(term_list_week_15), set(term_list_week_16))
words_rest_prior_12 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                 set(term_list_week_11))
words_rest_after_12 = set(term_list_week_13).union(set(term_list_week_14), 
                                                 set(term_list_week_15), set(term_list_week_16))

# Words - Week 13
words_week_13 = set(term_list_week_13)
words_rest_not_13 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), set(term_list_week_12), set(term_list_week_14),
                    set(term_list_week_15), set(term_list_week_16))
words_rest_prior_13 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                 set(term_list_week_11), set(term_list_week_12))
words_rest_after_13 = set(term_list_week_14).union(set(term_list_week_15), set(term_list_week_16))

# Words - Week 14
words_week_14 = set(term_list_week_14)
words_rest_not_14 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), set(term_list_week_12), set(term_list_week_13),
                    set(term_list_week_15), set(term_list_week_16))
words_rest_prior_14 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                 set(term_list_week_11), set(term_list_week_12), set(term_list_week_13))
words_rest_after_14 = set(term_list_week_15).union(set(term_list_week_16))

# Words - Week 15
words_week_15 = set(term_list_week_15)
words_rest_not_15 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                    set(term_list_week_16))
words_rest_prior_15 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), 
                                                 set(term_list_week_5), set(term_list_week_6), set(term_list_week_7),
                                                 set(term_list_week_8), set(term_list_week_9), set(term_list_week_10),
                                                 set(term_list_week_11), set(term_list_week_12), set(term_list_week_13))
words_rest_after_15 = set(term_list_week_16)

# Words - Week 16
words_week_16 = set(term_list_week_16)
words_rest_not_16 = set(term_list_week_1).union(set(term_list_week_2), set(term_list_week_3), set(term_list_week_4), set(term_list_week_5), set(term_list_week_6), set(term_list_week_7), set(term_list_week_8),
                    set(term_list_week_9), set(term_list_week_10), set(term_list_week_11), set(term_list_week_12), set(term_list_week_13), set(term_list_week_14),
                    set(term_list_week_15))
words_rest_prior_16 = words_rest_not_16

In [87]:
words_unique_to_week_1 = words_week_1.difference(words_rest_not_1)
words_unique_to_week_1_df = pd.DataFrame(words_unique_to_week_1)
words_unique_to_week_1_frequency_df = sorted_freq_dict_week_1_df[sorted_freq_dict_week_1_df['Word'].isin(words_unique_to_week_1)]

words_unique_to_week_2 = words_week_2.difference(words_rest_not_2)
words_unique_to_week_2_df = pd.DataFrame(words_unique_to_week_2)
words_unique_to_week_2_frequency_df = sorted_freq_dict_week_2_df[sorted_freq_dict_week_2_df['Word'].isin(words_unique_to_week_2)]

words_unique_to_week_3 = words_week_3.difference(words_rest_not_3)
words_unique_to_week_3_df = pd.DataFrame(words_unique_to_week_3)
words_unique_to_week_3_frequency_df = sorted_freq_dict_week_3_df[sorted_freq_dict_week_3_df['Word'].isin(words_unique_to_week_3)]

words_unique_to_week_4 = words_week_4.difference(words_rest_not_4)
words_unique_to_week_4_df = pd.DataFrame(words_unique_to_week_4)
words_unique_to_week_4_frequency_df = sorted_freq_dict_week_4_df[sorted_freq_dict_week_4_df['Word'].isin(words_unique_to_week_4)]

words_unique_to_week_5 = words_week_5.difference(words_rest_not_5)
words_unique_to_week_5_df = pd.DataFrame(words_unique_to_week_5)
words_unique_to_week_5_frequency_df = sorted_freq_dict_week_5_df[sorted_freq_dict_week_5_df['Word'].isin(words_unique_to_week_5)]

words_unique_to_week_6 = words_week_6.difference(words_rest_not_6)
words_unique_to_week_6_df = pd.DataFrame(words_unique_to_week_6)
words_unique_to_week_6_frequency_df = sorted_freq_dict_week_6_df[sorted_freq_dict_week_6_df['Word'].isin(words_unique_to_week_6)]

words_unique_to_week_7 = words_week_7.difference(words_rest_not_7)
words_unique_to_week_7_df = pd.DataFrame(words_unique_to_week_7)
words_unique_to_week_7_frequency_df = sorted_freq_dict_week_7_df[sorted_freq_dict_week_7_df['Word'].isin(words_unique_to_week_7)]

words_unique_to_week_8 = words_week_8.difference(words_rest_not_8)
words_unique_to_week_8_df = pd.DataFrame(words_unique_to_week_8)
words_unique_to_week_8_frequency_df = sorted_freq_dict_week_8_df[sorted_freq_dict_week_8_df['Word'].isin(words_unique_to_week_8)]

words_unique_to_week_9 = words_week_9.difference(words_rest_not_9)
words_unique_to_week_9_df = pd.DataFrame(words_unique_to_week_9)
words_unique_to_week_9_frequency_df = sorted_freq_dict_week_9_df[sorted_freq_dict_week_9_df['Word'].isin(words_unique_to_week_9)]

words_unique_to_week_10 = words_week_10.difference(words_rest_not_10)
words_unique_to_week_10_df = pd.DataFrame(words_unique_to_week_10)
words_unique_to_week_10_frequency_df = sorted_freq_dict_week_10_df[sorted_freq_dict_week_10_df['Word'].isin(words_unique_to_week_10)]

words_unique_to_week_11 = words_week_11.difference(words_rest_not_11)
words_unique_to_week_11_df = pd.DataFrame(words_unique_to_week_11)
words_unique_to_week_11_frequency_df = sorted_freq_dict_week_11_df[sorted_freq_dict_week_11_df['Word'].isin(words_unique_to_week_11)]

words_unique_to_week_12 = words_week_12.difference(words_rest_not_12)
words_unique_to_week_12_df = pd.DataFrame(words_unique_to_week_12)
words_unique_to_week_12_frequency_df = sorted_freq_dict_week_12_df[sorted_freq_dict_week_12_df['Word'].isin(words_unique_to_week_12)]

words_unique_to_week_13 = words_week_13.difference(words_rest_not_13)
words_unique_to_week_13_df = pd.DataFrame(words_unique_to_week_13)
words_unique_to_week_13_frequency_df = sorted_freq_dict_week_13_df[sorted_freq_dict_week_13_df['Word'].isin(words_unique_to_week_13)]

words_unique_to_week_14 = words_week_14.difference(words_rest_not_14)
words_unique_to_week_14_df = pd.DataFrame(words_unique_to_week_14)
words_unique_to_week_14_frequency_df = sorted_freq_dict_week_14_df[sorted_freq_dict_week_14_df['Word'].isin(words_unique_to_week_14)]

words_unique_to_week_15 = words_week_15.difference(words_rest_not_15)
words_unique_to_week_15_df = pd.DataFrame(words_unique_to_week_15)
words_unique_to_week_15_frequency_df = sorted_freq_dict_week_15_df[sorted_freq_dict_week_15_df['Word'].isin(words_unique_to_week_15)]

words_unique_to_week_16 = words_week_16.difference(words_rest_not_16)
words_unique_to_week_16_df = pd.DataFrame(words_unique_to_week_16)
words_unique_to_week_16_frequency_df = sorted_freq_dict_week_16_df[sorted_freq_dict_week_16_df['Word'].isin(words_unique_to_week_16)]

In [126]:
print('Number of words unique to week 1: ', words_unique_to_week_1_frequency_df.shape[0])
words_unique_to_week_1_frequency_df.head()

Number of words unique to week 1:  189


,Word,Frequency_Week_1,Rank_Week_1
217,qinghai,7,218
218,premier,7,219
281,hamster,6,282
293,bieng,6,294
334,mitten,5,335


In [125]:
words_unique_to_week_1_frequency_df = words_unique_to_week_1_frequency_df[['Word','Frequency_Week_1','Rank_Week_1']]
words_unique_to_week_1_frequency_df.to_csv('words_unique_to_week_1_frequency_rank.csv', encoding='utf-8',index=False)

In [123]:
print('Number of words unique to week 2: ', words_unique_to_week_2_frequency_df.shape[0])
words_unique_to_week_2_frequency_df.head()

Number of words unique to week 2:  437


,Frequency_Week_2,Word,Rank_Week_2
127,27,pinned,128
302,10,virusdr,303
309,10,publicand,310
348,9,involvement,349
416,7,realising,417


In [124]:
words_unique_to_week_2_frequency_df = words_unique_to_week_2_frequency_df[['Word','Frequency_Week_2','Rank_Week_2']]
words_unique_to_week_2_frequency_df.to_csv('words_unique_to_week_2_frequency_rank.csv', encoding='utf-8',index=False)

In [121]:
print('Number of words unique to week 3: ', words_unique_to_week_3_frequency_df.shape[0])
words_unique_to_week_3_frequency_df.head()

Number of words unique to week 3:  233


,Frequency_Week_3,Word,Rank_Week_3
105,16,bayonne,106
115,14,royal,116
132,13,caribbean,133
201,9,luggage,202
214,9,baggage,215


In [122]:
words_unique_to_week_3_frequency_df = words_unique_to_week_3_frequency_df[['Word','Frequency_Week_3','Rank_Week_3']]
words_unique_to_week_3_frequency_df.to_csv('words_unique_to_week_3_frequency_rank.csv', encoding='utf-8',index=False)

In [119]:
print('Number of words unique to week 4: ', words_unique_to_week_4_frequency_df.shape[0])
words_unique_to_week_4_frequency_df.head()

Number of words unique to week 4:  244


,Frequency_Week_4,Word,Rank_Week_4
43,37,playground,44
78,25,ripple,79
176,11,foothold,177
290,6,vpns,291
314,6,ontheground,315


In [120]:
words_unique_to_week_4_frequency_df = words_unique_to_week_4_frequency_df[['Word','Frequency_Week_4','Rank_Week_4']]
words_unique_to_week_4_frequency_df.to_csv('words_unique_to_week_4_frequency_rank.csv', encoding='utf-8',index=False)

In [118]:
print('Number of words unique to week 5: ', words_unique_to_week_5_frequency_df.shape[0])
words_unique_to_week_5_frequency_df.head()

Number of words unique to week 5:  203


,Word,Frequency_Week_5,Rank_Week_5
50,ifwhen,50,51
74,professionalshospitals,29,75
135,passcode,12,136
241,woth,6,242
244,texan,6,245


In [117]:
words_unique_to_week_5_frequency_df = words_unique_to_week_5_frequency_df[['Word','Frequency_Week_5','Rank_Week_5']]
words_unique_to_week_5_frequency_df.to_csv('words_unique_to_week_5_frequency_rank.csv', encoding='utf-8',index=False)

In [ ]:
print('Number of words unique to week 6: ', words_unique_to_week_6_frequency_df.shape[0])
words_unique_to_week_6_frequency_df.head()

In [116]:
words_unique_to_week_6_frequency_df = words_unique_to_week_6_frequency_df[['Word','Frequency_Week_6','Rank_Week_6']]
words_unique_to_week_6_frequency_df.to_csv('words_unique_to_week_6_frequency_rank.csv', encoding='utf-8',index=False)

In [88]:
print('Number of words unique to week 7: ', words_unique_to_week_7_frequency_df.shape[0])
words_unique_to_week_7_frequency_df.head()

Number of words unique to week 7:  847


,Frequency_Week_7,Word,Rank_Week_7
336,23,coronavirustravelersfaqshtml,337
391,18,insanest,392
392,18,headlining,393
502,13,precautionsn95,503
611,10,qatar,612


In [89]:
words_unique_to_week_7_frequency_df = words_unique_to_week_7_frequency_df[['Word','Frequency_Week_7','Rank_Week_7']]
words_unique_to_week_7_frequency_df.to_csv('words_unique_to_week_7_frequency_rank.csv', encoding='utf-8',index=False)

In [90]:
print('Number of words unique to week 8: ', words_unique_to_week_8_frequency_df.shape[0])
words_unique_to_week_8_frequency_df.head()

Number of words unique to week 8:  194


,Frequency_Week_8,Word,Rank_Week_8
160,8,soapwater,161
289,5,fvcking,290
360,4,gohmert,361
367,4,exterminate,368
404,3,unnerve,405


In [91]:
words_unique_to_week_8_frequency_df = words_unique_to_week_8_frequency_df[['Word','Frequency_Week_8','Rank_Week_8']]
words_unique_to_week_8_frequency_df.to_csv('words_unique_to_week_8_frequency_rank.csv', encoding='utf-8',index=False)

In [92]:
print('Number of words unique to week 9: ', words_unique_to_week_9_frequency_df.shape[0])
words_unique_to_week_9_frequency_df.head()

Number of words unique to week 9:  383


,Frequency_Week_9,Word,Rank_Week_9
60,67,loosened,61
102,36,diplomacy,103
104,36,ally,105
227,15,h1n1pdm09,228
295,12,bwcs2020,296


In [93]:
words_unique_to_week_9_frequency_df = words_unique_to_week_9_frequency_df[['Word','Frequency_Week_9','Rank_Week_9']]
words_unique_to_week_9_frequency_df.to_csv('words_unique_to_week_9_frequency_rank.csv', encoding='utf-8',index=False)

In [94]:
print('Number of words unique to week 10: ', words_unique_to_week_10_frequency_df.shape[0])
words_unique_to_week_10_frequency_df.head()

Number of words unique to week 10:  443


,Frequency_Week_10,Word,Rank_Week_10
113,45,sporadic,114
268,17,selfstyled,269
375,12,distracted,376
412,11,denies,413
563,7,swell,564


In [95]:
words_unique_to_week_10_frequency_df = words_unique_to_week_10_frequency_df[['Word','Frequency_Week_10','Rank_Week_10']]
words_unique_to_week_10_frequency_df.to_csv('words_unique_to_week_10_frequency_rank.csv', encoding='utf-8',index=False)

In [96]:
print('Number of words unique to week 11: ', words_unique_to_week_11_frequency_df.shape[0])
words_unique_to_week_11_frequency_df.head()

Number of words unique to week 11:  1489


,Frequency_Week_11,Word,Rank_Week_11
520,30,wasdo,521
541,29,rebel,542
557,28,determines,558
599,25,mulling,600
670,22,neera,671


In [97]:
words_unique_to_week_11_frequency_df = words_unique_to_week_11_frequency_df[['Word','Frequency_Week_11','Rank_Week_11']]
words_unique_to_week_11_frequency_df.to_csv('words_unique_to_week_11_frequency_rank.csv', encoding='utf-8',index=False)

In [98]:
print('Number of words unique to week 12: ', words_unique_to_week_12_frequency_df.shape[0])
words_unique_to_week_12_frequency_df.head()

Number of words unique to week 12:  698


,Frequency_Week_12,Word,Rank_Week_12
200,47,longtime,201
396,18,wyoming,397
476,15,dual,477
484,14,wield,485
490,14,sewistscosplayers,491


In [99]:
words_unique_to_week_12_frequency_df = words_unique_to_week_12_frequency_df[['Word','Frequency_Week_12','Rank_Week_12']]
words_unique_to_week_12_frequency_df.to_csv('words_unique_to_week_12_frequency_rank.csv', encoding='utf-8',index=False)

In [100]:
print('Number of words unique to week 13: ', words_unique_to_week_13_frequency_df.shape[0])
words_unique_to_week_13_frequency_df.head()

Number of words unique to week 13:  454


,Frequency_Week_13,Word,Rank_Week_13
115,51,btch,116
144,42,avidly,145
211,26,wray,212
376,12,skydiver,377
380,12,parachute,381


In [101]:
words_unique_to_week_13_frequency_df = words_unique_to_week_13_frequency_df[['Word','Frequency_Week_13','Rank_Week_13']]
words_unique_to_week_13_frequency_df.to_csv('words_unique_to_week_13_frequency_rank.csv', encoding='utf-8',index=False)

In [102]:
print('Number of words unique to week 14: ', words_unique_to_week_14_frequency_df.shape[0])
words_unique_to_week_14_frequency_df.head()

Number of words unique to week 14:  418


,Frequency_Week_14,Word,Rank_Week_14
10,100,baseline,11
23,75,genpop,24
24,75,covidcarrier,25
113,25,transission,114
117,24,pue,118


In [103]:
words_unique_to_week_14_frequency_df = words_unique_to_week_14_frequency_df[['Word','Frequency_Week_14','Rank_Week_14']]
words_unique_to_week_14_frequency_df.to_csv('words_unique_to_week_14_frequency_rank.csv', encoding='utf-8',index=False)

In [104]:
print('Number of words unique to week 15: ', words_unique_to_week_15_frequency_df.shape[0])
words_unique_to_week_15_frequency_df.head()

Number of words unique to week 15:  625


,Frequency_Week_15,Word,Rank_Week_15
152,37,menace,153
153,37,ingrahams,154
154,37,framing,155
353,14,yoda,354
444,11,ponders,445


In [105]:
words_unique_to_week_15_frequency_df = words_unique_to_week_15_frequency_df[['Word','Frequency_Week_15','Rank_Week_15']]
words_unique_to_week_15_frequency_df.to_csv('words_unique_to_week_15_frequency_rank.csv', encoding='utf-8',index=False)

In [108]:
print('Number of words unique to week 16: ', words_unique_to_week_16_frequency_df.shape[0])
words_unique_to_week_16_frequency_df.head()

Number of words unique to week 16:  515


,Word,Frequency_Week_16,Rank_Week_16
109,superstar,31,110
138,readjusted,26,139
263,concluding,15,264
289,tactical,13,290
307,firearm,13,308


In [107]:
words_unique_to_week_16_frequency_df = words_unique_to_week_16_frequency_df[['Word','Frequency_Week_16','Rank_Week_16']]
words_unique_to_week_16_frequency_df.to_csv('words_unique_to_week_16_frequency_rank.csv', encoding='utf-8',index=False)

In [ ]:
# cluster words by themselves (without the weights, or with weights being total frequency)
# vs. unsupervised (LDA, LDA for tweets / any shorter texts; paper recommended by Shagun: )

# other clusters: by culture (American vs. not), by geography (American vs. not)
# Memorial day, July 4th words - food/family gatherings?

# Trust over time! (trust related topics - subsets of lexicon)

In [ ]:
# positivity and domestic